# Capítulo 3: Selection Vectors

## Introdução

Os **Selection Vectors** são uma das otimizações mais elegantes do DuckDB. Eles permitem filtrar dados **sem copiar** - apenas mantendo uma lista de índices válidos.

### Objetivos deste capítulo:
1. Entender o que são Selection Vectors
2. Comparar filtragem tradicional vs Selection Vectors
3. Implementar Selection Vectors em Python
4. Medir o impacto na performance

In [ ]:
# Instalação das dependências
!pip install duckdb pandas numpy matplotlib seaborn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from typing import List, Optional, Tuple
from dataclasses import dataclass

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 3.1 O Problema da Filtragem Tradicional

Na abordagem tradicional, ao filtrar dados, criamos uma **cópia** dos dados que passam no filtro:

```
Dados Originais:          [10, 25, 15, 30, 8, 40, 12, 35]
Filtro: valor > 18
                            ↓
Dados Copiados (NOVO):    [25, 30, 40, 35]  ← Alocação de memória!
```

**Problemas**:
1. **Cópia de dados**: Gasta CPU movendo bytes
2. **Alocação de memória**: Cria novos arrays
3. **Cache pollution**: Dados novos podem "expulsar" dados úteis do cache

In [ ]:
# Demonstração: Filtragem tradicional com cópia
def filter_with_copy(data: np.ndarray, threshold: int) -> np.ndarray:
    """Filtra dados criando uma cópia (abordagem tradicional)"""
    mask = data > threshold
    return data[mask].copy()  # Cria novo array!

# Exemplo
original_data = np.array([10, 25, 15, 30, 8, 40, 12, 35], dtype=np.int64)
filtered_data = filter_with_copy(original_data, 18)

print("Dados originais:", original_data)
print(f"Endereço na memória: {original_data.ctypes.data}")
print(f"\nDados filtrados (> 18): {filtered_data}")
print(f"Endereço na memória: {filtered_data.ctypes.data}")
print("\n→ Endereços diferentes = memória diferente = CÓPIA!")

## 3.2 Selection Vectors: Filtragem Sem Cópia

A solução do DuckDB: **manter os dados originais** e apenas anotar quais índices são válidos!

```
Dados Originais (INALTERADO): [10, 25, 15, 30, 8, 40, 12, 35]
                               0   1   2   3   4   5   6   7

Filtro: valor > 18
                 ↓
Selection Vector:            [1, 3, 5, 7]  ← Apenas índices!

Para acessar dados filtrados:
  dados[sel_vec[0]] = dados[1] = 25
  dados[sel_vec[1]] = dados[3] = 30
  dados[sel_vec[2]] = dados[5] = 40
  dados[sel_vec[3]] = dados[7] = 35
```

**Vantagens**:
1. **Zero cópia de dados**: Dados originais intactos
2. **Mínima alocação**: Apenas um pequeno array de índices
3. **Cache friendly**: Dados originais permanecem no cache

In [ ]:
@dataclass
class SelectionVector:
    """Selection Vector do DuckDB"""
    indices: np.ndarray  # Array de índices válidos
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        return self.indices[idx]


class VectorWithSelection:
    """Vetor de dados com Selection Vector opcional"""
    def __init__(self, data: np.ndarray, selection: Optional[SelectionVector] = None):
        self.data = data
        self.selection = selection
    
    def get_value(self, idx: int):
        """Obtém valor considerando o Selection Vector"""
        if self.selection is not None:
            # Usa indireção através do selection vector
            actual_idx = self.selection[idx]
            return self.data[actual_idx]
        return self.data[idx]
    
    def get_count(self) -> int:
        """Retorna número de elementos válidos"""
        if self.selection is not None:
            return len(self.selection)
        return len(self.data)
    
    def get_all_values(self) -> List:
        """Retorna todos os valores válidos"""
        return [self.get_value(i) for i in range(self.get_count())]


def create_selection_vector(data: np.ndarray, condition) -> SelectionVector:
    """Cria Selection Vector baseado em uma condição"""
    mask = condition(data)
    indices = np.where(mask)[0].astype(np.uint32)
    return SelectionVector(indices)


print("Classes definidas!")

In [ ]:
# Demonstração: Filtragem com Selection Vector
data = np.array([10, 25, 15, 30, 8, 40, 12, 35], dtype=np.int64)

# Criar selection vector para valores > 18
sel_vec = create_selection_vector(data, lambda x: x > 18)

# Criar vetor com selection
filtered_view = VectorWithSelection(data, sel_vec)

print("Dados originais:", data)
print(f"Endereço dos dados: {data.ctypes.data}")
print(f"\nSelection Vector: {sel_vec.indices}")
print(f"Número de elementos selecionados: {filtered_view.get_count()}")
print(f"\nValores filtrados (via Selection Vector): {filtered_view.get_all_values()}")
print(f"\nEndereço dos dados originais (INALTERADO): {filtered_view.data.ctypes.data}")
print("\n→ Mesmo endereço = mesmos dados = ZERO CÓPIA!")

## 3.3 Visualização: Filtragem Tradicional vs Selection Vector

In [ ]:
def visualize_filtering(data, threshold):
    """Visualiza as duas abordagens de filtragem"""
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    mask = data > threshold
    
    # Tradicional (cópia)
    ax1 = axes[0]
    colors1 = ['#27ae60' if m else '#e74c3c' for m in mask]
    bars1 = ax1.bar(range(len(data)), data, color=colors1, edgecolor='black')
    ax1.axhline(y=threshold, color='blue', linestyle='--', label=f'Threshold = {threshold}')
    ax1.set_title('Abordagem Tradicional: Copia dados que passam no filtro', fontsize=12)
    ax1.set_xlabel('Índice')
    ax1.set_ylabel('Valor')
    
    # Adicionar setas mostrando cópia
    copied_data = data[mask]
    for i, (idx, val) in enumerate(zip(np.where(mask)[0], copied_data)):
        ax1.annotate('', xy=(len(data) + 1 + i*0.8, val), xytext=(idx, val),
                    arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
    
    # Mostrar dados copiados
    for i, val in enumerate(copied_data):
        ax1.bar(len(data) + 1 + i*0.8, val, width=0.6, color='#27ae60', 
               edgecolor='black', alpha=0.7)
    
    ax1.text(len(data) + 2, max(data) * 0.9, 'CÓPIA\n(nova memória)', 
            ha='center', fontsize=10, color='green')
    ax1.legend()
    
    # Selection Vector (sem cópia)
    ax2 = axes[1]
    colors2 = ['#27ae60' if m else '#bdc3c7' for m in mask]
    bars2 = ax2.bar(range(len(data)), data, color=colors2, edgecolor='black')
    ax2.axhline(y=threshold, color='blue', linestyle='--', label=f'Threshold = {threshold}')
    ax2.set_title('Selection Vector: Apenas anota índices válidos', fontsize=12)
    ax2.set_xlabel('Índice')
    ax2.set_ylabel('Valor')
    
    # Mostrar selection vector
    sel_indices = np.where(mask)[0]
    sel_text = f"Selection Vector: {list(sel_indices)}"
    ax2.text(len(data)/2, max(data) * 1.1, sel_text, ha='center', fontsize=12,
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    # Destacar índices selecionados
    for idx in sel_indices:
        ax2.annotate(f'[{idx}]', xy=(idx, data[idx]), xytext=(idx, data[idx] + 3),
                    ha='center', fontsize=9, fontweight='bold', color='green')
    
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Visualizar
visualize_filtering(np.array([10, 25, 15, 30, 8, 40, 12, 35]), 18)

## 3.4 Benchmark: Performance de Filtragem

In [ ]:
def benchmark_filtering(size: int, selectivity: float, n_iterations: int = 100):
    """
    Compara performance de filtragem com cópia vs Selection Vector.
    
    Args:
        size: Tamanho do array
        selectivity: Fração de elementos que passam no filtro (0.0 a 1.0)
        n_iterations: Número de iterações para média
    """
    # Criar dados de teste
    data = np.random.randint(0, 100, size=size, dtype=np.int64)
    threshold = int(100 * (1 - selectivity))  # Ajusta threshold para atingir selectivity
    
    # Benchmark: Cópia tradicional
    copy_times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        filtered = data[data > threshold].copy()
        copy_times.append(time.perf_counter() - start)
    
    # Benchmark: Selection Vector
    sel_times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        indices = np.where(data > threshold)[0].astype(np.uint32)
        sel_times.append(time.perf_counter() - start)
    
    return {
        'copy_mean': np.mean(copy_times) * 1000,  # ms
        'copy_std': np.std(copy_times) * 1000,
        'sel_mean': np.mean(sel_times) * 1000,
        'sel_std': np.std(sel_times) * 1000,
        'actual_selectivity': np.mean(data > threshold)
    }

# Rodar benchmarks com diferentes seletividades
sizes = [10_000, 100_000, 1_000_000]
selectivities = [0.1, 0.3, 0.5, 0.7, 0.9]

results = []
for size in sizes:
    for sel in selectivities:
        r = benchmark_filtering(size, sel)
        r['size'] = size
        r['selectivity'] = sel
        results.append(r)
        print(f"Size: {size:>10,} | Selectivity: {sel:.0%} | "
              f"Copy: {r['copy_mean']:.3f}ms | Selection: {r['sel_mean']:.3f}ms")

In [ ]:
# Visualização dos resultados
df_results = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Tempo por seletividade (1M elementos)
ax1 = axes[0]
df_1m = df_results[df_results['size'] == 1_000_000]
x = np.arange(len(selectivities))
width = 0.35

ax1.bar(x - width/2, df_1m['copy_mean'], width, label='Cópia', color='#e74c3c', yerr=df_1m['copy_std'])
ax1.bar(x + width/2, df_1m['sel_mean'], width, label='Selection Vector', color='#27ae60', yerr=df_1m['sel_std'])
ax1.set_xlabel('Seletividade')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Tempo de Filtragem (1M elementos)')
ax1.set_xticks(x)
ax1.set_xticklabels([f'{s:.0%}' for s in selectivities])
ax1.legend()

# Gráfico 2: Speedup
ax2 = axes[1]
df_results['speedup'] = df_results['copy_mean'] / df_results['sel_mean']

for size in sizes:
    df_size = df_results[df_results['size'] == size]
    ax2.plot(df_size['selectivity'] * 100, df_size['speedup'], 
             marker='o', label=f'{size:,} elementos')

ax2.axhline(y=1, color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('Seletividade (%)')
ax2.set_ylabel('Speedup (Cópia / Selection Vector)')
ax2.set_title('Speedup do Selection Vector')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.5 Encadeamento de Filtros com Selection Vectors

Uma grande vantagem dos Selection Vectors é que filtros podem ser **encadeados** sem nunca copiar dados!

In [ ]:
def chain_filters_with_selection(data: np.ndarray, *conditions) -> VectorWithSelection:
    """
    Aplica múltiplos filtros encadeados usando Selection Vectors.
    Cada filtro subsequente trabalha apenas nos índices válidos.
    """
    # Inicialmente, todos os índices são válidos
    current_indices = np.arange(len(data), dtype=np.uint32)
    
    for i, condition in enumerate(conditions):
        # Aplica condição apenas nos dados dos índices atuais
        current_data = data[current_indices]
        mask = condition(current_data)
        
        # Atualiza índices (mantém apenas os que passaram)
        current_indices = current_indices[mask]
        
        print(f"Após filtro {i+1}: {len(current_indices)} elementos restantes")
    
    return VectorWithSelection(data, SelectionVector(current_indices))

# Exemplo: Encadeamento de 3 filtros
data = np.random.randint(0, 100, size=10000, dtype=np.int64)

print(f"Dados originais: {len(data)} elementos")
print("\nAplicando filtros encadeados:")

result = chain_filters_with_selection(
    data,
    lambda x: x > 20,    # Filtro 1: > 20
    lambda x: x < 80,    # Filtro 2: < 80
    lambda x: x % 2 == 0 # Filtro 3: números pares
)

print(f"\nResultado final: {result.get_count()} elementos")
print(f"Primeiros 10 valores: {result.get_all_values()[:10]}")
print(f"\n→ Dados originais NUNCA foram copiados!")

## 3.6 Operações Vetorizadas com Selection Vectors

Operações aritméticas podem ser otimizadas usando Selection Vectors.

In [ ]:
def vectorized_operation_with_selection(
    vec: VectorWithSelection, 
    operation
) -> np.ndarray:
    """
    Aplica operação apenas nos elementos selecionados.
    Muito mais eficiente quando poucos elementos estão selecionados!
    """
    if vec.selection is None:
        # Sem selection, opera em todos
        return operation(vec.data)
    else:
        # Com selection, opera apenas nos índices válidos
        result = np.zeros_like(vec.data)
        selected_indices = vec.selection.indices
        result[selected_indices] = operation(vec.data[selected_indices])
        return result

# Comparação: Operação com e sem Selection Vector
size = 1_000_000
data = np.random.randn(size)

# Criar selection com 10% dos dados
sel_indices = np.random.choice(size, size=size//10, replace=False).astype(np.uint32)
sel_vec = SelectionVector(np.sort(sel_indices))

vec_full = VectorWithSelection(data)
vec_selected = VectorWithSelection(data, sel_vec)

# Operação: calcular exp(x)
operation = lambda x: np.exp(x)

# Benchmark sem selection
start = time.perf_counter()
for _ in range(10):
    result_full = vectorized_operation_with_selection(vec_full, operation)
time_full = (time.perf_counter() - start) / 10

# Benchmark com selection
start = time.perf_counter()
for _ in range(10):
    result_selected = vectorized_operation_with_selection(vec_selected, operation)
time_selected = (time.perf_counter() - start) / 10

print(f"Operação exp(x) em {size:,} elementos:")
print(f"  Sem Selection Vector: {time_full*1000:.2f} ms")
print(f"  Com Selection Vector (10%): {time_selected*1000:.2f} ms")
print(f"  Speedup: {time_full/time_selected:.1f}x")

## 3.7 DuckDB em Ação: Observando Selection Vectors

In [ ]:
# Conectar ao DuckDB
con = duckdb.connect(':memory:')

# Criar tabela de teste
con.execute("""
    CREATE TABLE vendas AS
    SELECT 
        i AS id,
        random() * 1000 AS valor,
        CASE (i % 5)
            WHEN 0 THEN 'Eletrônicos'
            WHEN 1 THEN 'Roupas'
            WHEN 2 THEN 'Alimentos'
            WHEN 3 THEN 'Móveis'
            ELSE 'Outros'
        END AS categoria,
        CASE WHEN random() > 0.7 THEN 'Premium' ELSE 'Regular' END AS tipo_cliente
    FROM generate_series(1, 1000000) AS t(i)
""")

print("Tabela criada com 1.000.000 de linhas")
con.execute("SELECT * FROM vendas LIMIT 5").df()

In [ ]:
# Consulta com múltiplos filtros
query = """
    SELECT categoria, AVG(valor) as media_valor
    FROM vendas
    WHERE valor > 500
      AND tipo_cliente = 'Premium'
    GROUP BY categoria
    ORDER BY media_valor DESC
"""

# Ver plano de execução
print("=== PLANO DE EXECUÇÃO ===")
plan = con.execute(f"EXPLAIN {query}").df()
print(plan.to_string())

print("\n=== RESULTADO ===")
result = con.execute(query).df()
print(result)

In [ ]:
# Analisar com EXPLAIN ANALYZE
print("=== EXPLAIN ANALYZE ===")
analyze = con.execute(f"EXPLAIN ANALYZE {query}").df()
print(analyze.to_string())

## 3.8 Impacto da Seletividade na Performance

In [ ]:
# Benchmark DuckDB com diferentes seletividades
def benchmark_duckdb_selectivity(con, threshold: int, n_runs: int = 10):
    """Mede tempo de consulta com diferentes thresholds"""
    query = f"SELECT SUM(valor) FROM vendas WHERE valor > {threshold}"
    
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    
    # Calcular seletividade real
    total = con.execute("SELECT COUNT(*) FROM vendas").fetchone()[0]
    selected = con.execute(f"SELECT COUNT(*) FROM vendas WHERE valor > {threshold}").fetchone()[0]
    selectivity = selected / total
    
    return {
        'threshold': threshold,
        'selectivity': selectivity,
        'time_ms': np.mean(times) * 1000
    }

# Testar diferentes thresholds
thresholds = [100, 300, 500, 700, 900]
duck_results = [benchmark_duckdb_selectivity(con, t) for t in thresholds]

print("\nResultados do Benchmark:")
for r in duck_results:
    print(f"Threshold: {r['threshold']:>4} | Seletividade: {r['selectivity']:.1%} | Tempo: {r['time_ms']:.2f}ms")

In [ ]:
# Visualização
fig, ax = plt.subplots(figsize=(10, 5))

selectivities = [r['selectivity'] * 100 for r in duck_results]
times = [r['time_ms'] for r in duck_results]

ax.plot(selectivities, times, marker='o', linewidth=2, markersize=8, color='#3498db')
ax.fill_between(selectivities, times, alpha=0.3, color='#3498db')

ax.set_xlabel('Seletividade (%)', fontsize=12)
ax.set_ylabel('Tempo (ms)', fontsize=12)
ax.set_title('DuckDB: Tempo de Consulta vs Seletividade do Filtro', fontsize=14)
ax.grid(True, alpha=0.3)

# Anotar pontos
for sel, t, r in zip(selectivities, times, duck_results):
    ax.annotate(f'{t:.1f}ms', xy=(sel, t), xytext=(5, 5), 
               textcoords='offset points', fontsize=9)

plt.tight_layout()
plt.show()

## 3.9 Exercícios Práticos

### Exercício 1: Implementar operação de Join usando Selection Vectors

In [ ]:
def semi_join_with_selection(
    left_data: np.ndarray,
    right_keys: set
) -> VectorWithSelection:
    """
    Implementa um Semi-Join usando Selection Vector.
    Retorna elementos de left_data cujas chaves estão em right_keys.
    """
    # Encontrar índices onde left_data está em right_keys
    mask = np.isin(left_data, list(right_keys))
    indices = np.where(mask)[0].astype(np.uint32)
    
    return VectorWithSelection(left_data, SelectionVector(indices))

# Teste
left = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
right_keys = {2, 4, 6, 8, 10}

result = semi_join_with_selection(left, right_keys)

print(f"Dados da esquerda: {left}")
print(f"Chaves da direita: {right_keys}")
print(f"Selection Vector: {result.selection.indices}")
print(f"Resultado do Semi-Join: {result.get_all_values()}")

### Exercício 2: Comparar memória usada

In [ ]:
def compare_memory_usage(size: int, selectivity: float):
    """Compara uso de memória entre cópia e Selection Vector"""
    
    # Criar dados
    data = np.random.randint(0, 100, size=size, dtype=np.int64)
    threshold = int(100 * (1 - selectivity))
    
    # Método 1: Cópia
    filtered_copy = data[data > threshold].copy()
    memory_copy = filtered_copy.nbytes
    
    # Método 2: Selection Vector
    indices = np.where(data > threshold)[0].astype(np.uint32)
    memory_selection = indices.nbytes  # Apenas os índices
    
    return {
        'original_memory': data.nbytes,
        'copy_memory': memory_copy,
        'selection_memory': memory_selection,
        'elements_selected': len(indices),
        'savings_ratio': memory_copy / memory_selection if memory_selection > 0 else float('inf')
    }

# Testar
print("COMPARAÇÃO DE USO DE MEMÓRIA")
print("="*60)

for selectivity in [0.1, 0.3, 0.5, 0.7, 0.9]:
    result = compare_memory_usage(1_000_000, selectivity)
    print(f"\nSeletividade: {selectivity:.0%}")
    print(f"  Elementos selecionados: {result['elements_selected']:,}")
    print(f"  Memória cópia: {result['copy_memory']/1024/1024:.2f} MB")
    print(f"  Memória Selection: {result['selection_memory']/1024/1024:.2f} MB")
    print(f"  Economia: {result['savings_ratio']:.1f}x menos memória")

## 3.10 Resumo do Capítulo

### Selection Vectors vs Filtragem Tradicional

| Aspecto | Filtragem Tradicional | Selection Vector |
|---------|----------------------|------------------|
| Dados | Copiados | Intactos |
| Memória extra | O(k) - dados filtrados | O(k) - apenas índices |
| Cache | Polui com novos dados | Mantém dados no cache |
| Encadeamento | Múltiplas cópias | Zero cópias |
| Melhor para | Alta seletividade | Baixa/média seletividade |

### Principais Takeaways:

1. **Selection Vectors evitam cópias** mantendo apenas índices válidos
2. **Filtros podem ser encadeados** sem nunca copiar dados
3. **Cache permanece eficiente** pois dados originais não são movidos
4. **Performance depende da seletividade** - mais eficiente com menos elementos
5. **Operações vetorizadas** podem pular elementos não selecionados

In [ ]:
# Limpar recursos
con.close()